In [ ]:

import cv2
import os
import numpy as np
from PIL import Image, ImageEnhance, ImageOps
import random


# Definir la ruta de la carpeta de imágenes y la carpeta de salida para las imágenes aumentadas
ruta_carpeta = '../Caras_cortadas'
ruta_salida = '../Caras_aumentadas'

In [2]:
import os
import cv2
import numpy as np

# Crear la carpeta de salida si no existe
if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)

# Funciones de aumentación
def rotar_imagen(imagen, angulo):
    """Rota la imagen en el ángulo especificado."""
    (h, w) = imagen.shape[:2]
    centro = (w // 2, h // 2)
    matriz_rotacion = cv2.getRotationMatrix2D(centro, angulo, 1.0)
    return cv2.warpAffine(imagen, matriz_rotacion, (w, h))

def desplazar_imagen(imagen, x, y):
    """Desplaza la imagen en x e y."""
    matriz_desplazamiento = np.float32([[1, 0, x], [0, 1, y]])
    return cv2.warpAffine(imagen, matriz_desplazamiento, (imagen.shape[1], imagen.shape[0]))

def escalar_imagen(imagen, escala):
    """Escala la imagen por el factor especificado."""
    h, w = imagen.shape[:2]
    return cv2.resize(imagen, (int(w * escala), int(h * escala)))

def aplicar_cutout(imagen, tamano_recorte):
    """Aplica un recorte negro (cutout) a la imagen en una ubicación aleatoria."""
    h, w = imagen.shape[:2]
    x_inicio = np.random.randint(0, max(1, w - tamano_recorte))
    y_inicio = np.random.randint(0, max(1, h - tamano_recorte))
    imagen_cutout = imagen.copy()
    imagen_cutout[y_inicio:y_inicio+tamano_recorte, x_inicio:x_inicio+tamano_recorte] = 0
    return imagen_cutout

def voltear_imagen(imagen, direccion):
    """Voltea la imagen: horizontal (1), vertical (0)."""
    return cv2.flip(imagen, direccion)

def transformar_perspectiva(imagen):
    """Aplica una transformación de perspectiva aleatoria."""
    h, w = imagen.shape[:2]
    puntos_originales = np.float32([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
    desplazamiento = np.random.randint(-10, 10, (4, 2)).astype(np.float32)
    puntos_destino = puntos_originales + desplazamiento
    matriz_perspectiva = cv2.getPerspectiveTransform(puntos_originales, puntos_destino)
    return cv2.warpPerspective(imagen, matriz_perspectiva, (w, h))

# Aplicar aumentaciones
for persona in os.listdir(ruta_carpeta):
    ruta_persona = os.path.join(ruta_carpeta, persona)
    if os.path.isdir(ruta_persona):
        ruta_salida_persona = os.path.join(ruta_salida, persona)
        if not os.path.exists(ruta_salida_persona):
            os.makedirs(ruta_salida_persona)
        
        for archivo in os.listdir(ruta_persona):
            ruta_imagen = os.path.join(ruta_persona, archivo)
            imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
            
            if imagen is not None:
                # Guardar la imagen original
                cv2.imwrite(os.path.join(ruta_salida_persona, archivo), imagen)

                # Rotaciones con ángulos pequeños y grandes
                angulos_pequenos = range(-15, 16, 3)  # Ángulos de -15 a 15 con pasos de 3
                angulos_grandes = [15, -15, 30, -30, 45, -45, 90, -90]
                angulos_totales = list(angulos_pequenos) + angulos_grandes

                for angulo in angulos_totales:
                    imagen_rotada = rotar_imagen(imagen, angulo)
                    nombre_archivo = f"{archivo.split('.')[0]}_rotada_{angulo}.jpg"
                    cv2.imwrite(os.path.join(ruta_salida_persona, nombre_archivo), imagen_rotada)
                
                # Desplazamientos
                for desplazamiento in [(5, 5), (-5, -5), (10, 0), (0, 10), (15, -10), (-10, 15)]:
                    imagen_desplazada = desplazar_imagen(imagen, desplazamiento[0], desplazamiento[1])
                    nombre_archivo = f"{archivo.split('.')[0]}_desplazada_{desplazamiento[0]}_{desplazamiento[1]}.jpg"
                    cv2.imwrite(os.path.join(ruta_salida_persona, nombre_archivo), imagen_desplazada)
                
                # Escalado
                for escala in [1.1, 0.9, 1.2, 0.8]:
                    imagen_escalada = escalar_imagen(imagen, escala)
                    nombre_archivo = f"{archivo.split('.')[0]}_escalada_{escala}.jpg"
                    cv2.imwrite(os.path.join(ruta_salida_persona, nombre_archivo), imagen_escalada)

                # Cutout
                for tamano in [5, 10, 15]:
                    imagen_cutout = aplicar_cutout(imagen, tamano)
                    nombre_archivo = f"{archivo.split('.')[0]}_cutout_{tamano}.jpg"
                    cv2.imwrite(os.path.join(ruta_salida_persona, nombre_archivo), imagen_cutout)

                # Volteos (solo horizontales y verticales)
                direccion=1  # 0: vertical, 1: horizontal
                imagen_volteada = voltear_imagen(imagen, direccion)
                nombre_archivo = f"{archivo.split('.')[0]}_flip_{direccion}.jpg"
                cv2.imwrite(os.path.join(ruta_salida_persona, nombre_archivo), imagen_volteada)
                
                # Transformación de perspectiva
                imagen_perspectiva = transformar_perspectiva(imagen)
                nombre_archivo = f"{archivo.split('.')[0]}_perspectiva.jpg"
                cv2.imwrite(os.path.join(ruta_salida_persona, nombre_archivo), imagen_perspectiva)

print("Aumentación de imágenes completada.")

Aumentación de imágenes completada.
